Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import os

import numpy as np
import pandas as pd
from scipy.special import expit
import tensorflow as tf

from bert import modeling
from bert import optimization
from bert import tokenization

import multilabel

In [ ]:
# Load BERT

GCP_BUCKET = "gs://ubert-overflow"
BERT_DIR = f"{GCP_BUCKET}/multi_cased_L-12_H-768_A-12/"
MAX_SEQ_LENGTH = 128

tokenization.validate_case_matches_checkpoint(False,
                                              BERT_DIR + "bert_config.json")
tokenizer = tokenization.FullTokenizer(BERT_DIR + "vocab.txt",
                                       do_lower_case=False)
bert_config = modeling.BertConfig.from_json_file(BERT_DIR + "bert_config.json")
if MAX_SEQ_LENGTH > bert_config.max_position_embeddings:
    raise ValueError("Cannot use sequence length %d because the BERT model "
                     "was only trained up to sequence length %d" %
                     (MAX_SEQ_LENGTH, bert_config.max_position_embeddings))

In [ ]:
# Get predictions on a list of sentences


def getPrediction(in_sentences, batch_size, checkpoint, use_tpu, sample_size=1):
    labels = multilabel.LABEL_COLUMN
    input_examples = [
        multilabel.InputExample(guid=i,
                                      text_a=in_sentences[i],
                                      text_b=None,
                                      label=[0] * len(labels),
                                       label_mask = [1] * len(labels))
        for i in range(len(in_sentences))
    ]
    input_examples *= sample_size
    num_actual_example = len(input_examples)
    if use_tpu:
        while num_actual_example % batch_size != 0:
            input_examples.append(multilabel.PaddingInputExample())
        assert num_actual_example % batch_size == 0
    input_features = multilabel.convert_examples_to_features(
        input_examples, labels, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = multilabel.input_fn_builder(
        features=input_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=use_tpu)
    predictions = estimator.predict(predict_input_fn, checkpoint_path=checkpoint)
    if sample_size > 1:
        probs = []
        logits = []
        per_iter_probs = []
        per_iter_logits = []
        for (i, pred) in enumerate(predictions):
            if i > 0 and i % len(in_sentences) == 0:
                probs.append(per_iter_probs)
                logits.append(per_iter_logits)
                per_iter_probs = []
                per_iter_logits = []
            if i >= num_actual_example:
                break
            per_iter_probs.append(pred["probs"])
            per_iter_logits.append(pred["logits"])
        probs = np.array(probs)
        logits = np.array(logits)
        prob = probs.mean(axis=0)
        variance = logits.var(axis=0)
        mean_logits = logits.mean(axis=0)
        predictions = [{
            "probs": prob[i],
            "variance": variance[i],
            "logits": mean_logits[i]
        } for i in range(len(prob))]

    results = []
    for sentence, prediction in zip(in_sentences, predictions):
        logits = np.array(prediction["logits"], dtype=np.float)
        vars = np.array(prediction['variance'], dtype=np.float)
        std_dev = np.sqrt(vars)
        lower_ci = expit(logits - std_dev)
        upper_ci = expit(logits + std_dev)
        lower_95_ci = expit(logits - 2 * std_dev)
        upper_95_ci = expit(logits + 2 * std_dev)
        
        df = pd.DataFrame(np.array(
            [prediction['probs'], vars, lower_ci, upper_ci, lower_95_ci, upper_95_ci, labels]).T,
                          columns=["probability", "logit_variance", "68% CI LB", "68% CI UB", "95% CI LB", "95% CI UB", "type"])
        df = df.astype({
            "probability": float,
            "logit_variance": float,
            "68% CI LB": float,
            "68% CI UB": float,
            "95% CI LB": float,
            "95% CI UB": float,
            "type": str
        })
        results.append((sentence, df))
    return results

In [ ]:
# Set up the estimator

FINETUNE_MODULE = "original"  # original, mc, bnn
CKPT = "best"    
USE_TPU = False
BATCH_SIZE = 256 if USE_TPU else 16
FREEZE_BERT = False
OUTPUT_DIR = f"{GCP_BUCKET}/output/{FINETUNE_MODULE}_20"
CKPT_PATH = f"{OUTPUT_DIR}/model.ckpt-{CKPT}"

tpu_cluster_resolver = None
if USE_TPU:
    os.system("gcloud compute tpus start fb-demo --zone=us-central1-b")
    tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        "fb-demo", zone="us-central1-b", project="kats-uncertainty-estimation")
is_per_host = tf.compat.v1.estimator.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.compat.v1.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    tpu_config=tf.compat.v1.estimator.tpu.TPUConfig(
        iterations_per_loop=1,
        num_shards=8,
        per_host_input_for_training=is_per_host),
    model_dir=OUTPUT_DIR)

model_fn = multilabel.model_fn_builder(
    bert_config=bert_config,
    num_labels=len(multilabel.LABEL_COLUMN),
    init_checkpoint=tf.train.latest_checkpoint(BERT_DIR),
    learning_rate=0,
    num_train_steps=0,
    num_warmup_steps=0,
    use_tpu=USE_TPU,
    use_one_hot_embeddings=USE_TPU,
    freeze_bert=FREEZE_BERT,
    finetune_module=FINETUNE_MODULE,
    num_train_examples=140215)

estimator = tf.estimator.tpu.TPUEstimator(use_tpu=USE_TPU,
                                          model_fn=model_fn,
                                          config=run_config,
                                          train_batch_size=BATCH_SIZE,
                                          eval_batch_size=BATCH_SIZE,
                                          predict_batch_size=BATCH_SIZE)

In [ ]:
pred_sentences = [
    "Jet Li is an actor.", "blue hawaii is a mountain.",
    "blue hawaii is a fictional mountain."
]
predictions = getPrediction(pred_sentences, BATCH_SIZE, CKPT_PATH, USE_TPU, sample_size=100)
pd.set_option('display.float_format', '{:.5f}'.format)
for item in predictions:
    print(
        f"{item[0]}\n{item[1].sort_values(by=['probability'],axis=0, ascending=False)}\n"
    )